Input data:
 - building footprints with height attribute
 - street network

This notebook generates morphometric elements required for further analysis. It requires `momepy` 0.3 or newer.

In [ ]:
import geopandas as gpd
import momepy as mm

In [ ]:
path = "files/geometry.gpkg"
layer = "buildings"

Open buildings and assing unique ID to each.

In [ ]:
buildings = gpd.read_file(path, layer=layer)
buildings["uID"] = range(len(buildings))

## Morphological tessellation

Check input for ressellation.

In [ ]:
check = mm.CheckTessellationInput(buildings)

Generate tessellation limited to 100 m buffer. Beware, it eats a ton of memory.

In [ ]:
limit = mm.buffered_limit(buildings, 100)

tess = mm.Tessellation(buildings, "uID", limit)
tessellation = tess.queen_corners(sensitivity=2) # this is a dirty fix I am not using anymore, but I did in thesis


In [ ]:
tessellation.to_file("files/geometry.gpkg", layer="tessellation", driver="GPKG")

## Tessellation based blocks

We need streets for that.

In [ ]:
streets = gpd.read_file('files/geometry.gpkg', layer='streets')

In [ ]:
snapped = mm.snap_street_network_edge(streets, buildings, 20, tessellation, 120, limit) # snap to close unwanted gaps
blocks = mm.Blocks(tessellation, snapped, buildings, 'bID', 'uID')
blocks_df = blocks.blocks  # get blocks df
buildings['bID'] = blocks.buildings_id  # get block ID
tessellation['bID'] = blocks.tessellation_id  # get block ID

## Link streets

In [ ]:
streets["nID"] = range(len(streets))
buildings['nID'] = mm.get_network_id(buildings, streets, 'nID', min_size=300)  # 
tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [ ]:
path = 'files/geometry.gpkg'
tessellation.to_file(path, layer='tessellation', driver='GPKG')
buildings.to_file(path, layer='buildings', driver='GPKG')
blocks_df.to_file(path, layer='blocks', driver='GPKG')